# Atlas Surface Rendering
Adam Saunders

[MASI Lab](https://my.vanderbilt.edu/masi/), Vanderbilt University

Often, when using atlases for volumetric data, we wish to see the deformation resulting from registering a subject to an atlas. Here, we present code for generating a VTK surface from a subject image. We superimpose a checkerboard that undergoes the inverse deformation from atlas space to subject space. We can compare this checkerboard across several subjects to gain an understanding of how consistent our registration is.

The process for generating the surface is:
1. Generate a checkerboard in atlas space
2. Apply the inverse deformations from atlas to subject space to the checkerboard
3. Create the surface with the checkerboard

The code allows us to customize the grid size and number of colors for our checkerboard. Additionally, we provide code to generate a custom color map for Paraview.

## Download and register data

For this example, we will use the T1-weighted [MNI152 template](https://nist.mni.mcgill.ca/icbm-152-nonlinear-atlases-2009/) as our atlas. We use the T1-weighted image from subject 17 from [OpenNeuro dataset 001233](https://github.com/OpenNeuroDatasets/ds001233).

You can also get the data directly in Python using the templateflow and ndslib packages to download the data:

In [ ]:
import templateflow.api as tflow
from ndslib.data import download_bids_dataset
import nibabel as nib
download_bids_dataset()
mni_img = nib.load(tflow.get('MNI152NLin2009cAsym', resolution=1, suffix='T1w', desc=None))
nib.save(mni_img, 'data/templates/MNI152NLin2009cAsym.nii')

First we need register our data. I used a [simple registration provided by the ANTs repository](https://github.com/ANTsX/ANTs/wiki/Anatomy-of-an-antsRegistration-call):

In [ ]:
%%bash
thisfolder="data/registration"
sub="sub-17"
template="data/subjects/sub-17_ses-pre_T1w.nii.gz"
t1brain="data/templates/MNI152NLin2009cAsym.nii"
antsRegistration --dimensionality 3 --float 0 \
    --output [$thisfolder/template_to_${sub}_,$thisfolder/template_to_${sub}_Warped.nii.gz] \
    --interpolation Linear \
    --winsorize-image-intensities [0.005,0.995] \
    --use-histogram-matching 0 \
    --initial-moving-transform [$t1brain,$template,1] \
    --transform Rigid[0.1] \
    --metric MI[$t1brain,$template,1,32,Regular,0.25] \
    --convergence [1000x500x250x100,1e-6,10] \
    --shrink-factors 8x4x2x1 \
    --smoothing-sigmas 3x2x1x0vox \
    --transform Affine[0.1] \
    --metric MI[$t1brain,$template,1,32,Regular,0.25] \
    --convergence [1000x500x250x100,1e-6,10] \
    --shrink-factors 8x4x2x1 \
    --smoothing-sigmas 3x2x1x0vox \
    --transform SyN[0.1,3,0] \
    --metric CC[$t1brain,$template,1,4] \
    --convergence [100x70x50x20,1e-6,10] \
    --shrink-factors 8x4x2x1 \
    --smoothing-sigmas 3x2x1x0vox

## Preprocess data

For the purpose of this exercise, we want to use atlas and subject labels instead of the MRI images. First, we perform skull stripping using BET and then segment the image using FAST, both available with the [FSL toolbox](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki).

In [ ]:
%%bash
bet data/subjects/sub-17_ses-pre_T1w.nii.gz data/subjects_brain/sub-17_ses-pre_T1w.nii.gz
bet data/templates/MNI152NLin2009cAsym.nii.gz data/templates_brain/MNI152NLin2009cAsym.nii.gz
fast -t 1 -n 3 -H 0.1 -I 4 -l 20.0 -g -o data/templates_brain/MNI152NLin2009cAsym data/templates_brain/MNI152NLin2009cAsym
fast -t 1 -n 3 -H 0.1 -I 4 -l 20.0 -g -o data/subjects_brain/sub-17_ses-pre_T1w data/subjects_brain/sub-17_ses-pre_T1w

Next, since most of the deformation appears along the inside of the brain, we will choose to generate a surface for half of the gray matter using a sagittal view. We create this ROI and copy the corresponding atlas label:

In [ ]:
%%bash
fslmaths data/subjects_brain/sub-17_ses-pre_T1w_seg_2.nii.gz -roi 96 -1 0 -1 0 -1 0 1 data/subjects_label/sub-17_ses-pre_T1w.nii.gz
cp data/templates_brain/MNI152NLin2009cAsym_seg_2.nii.gz data/templates_label/MNI152NLin2009cAsym.nii.gz

## 1. Generate checkerboard

First, we generate the checkerboard in atlas space. Note that we can customize the number of colors and the grid size, as well as the view (coronal, sagittal, axial). Here, we generate a sagittal view with a grid size of 16 and 12 colors. We name the checkerboard the same name as our subject since we will be deforming it into subject space. This is helpful when generating surfaces for many subjects.

In [81]:
%%bash
python checkerboard_generate.py \
    --input_label data/templates_label/MNI152NLin2009cAsym.nii.gz \
    --grid_size 16 \
    --view 2 \
    --output_cb data/checkerboard/sub-17_ses-pre_T1w.nii.gz \
    --num_colors 12

## 2. Apply inverse deformation to checkerboard

Next, we apply the inverse transformations from atlas space to subject space to our checkerboard.

In [82]:
%%bash
affine="data/registration/template_to_sub-17_0GenericAffine.mat"
invWarp="data/registration/template_to_sub-17_1InverseWarp.nii.gz"

antsApplyTransforms -d 3 -i data/checkerboard/sub-17_ses-pre_T1w.nii.gz \
    -r data/subjects_label/sub-17_ses-pre_T1w.nii.gz \
    -o data/checkerboard_deformed/sub-17_ses-pre_T1w.nii.gz \
    -n NearestNeighbor \
    -t [$affine,1] \
    -t $invWarp  

## 3. Generate surface

Finally, we generate a VTK surface from the subject with the deformed checkerboard superimposed. This file can be viewed in Paraview.

In [84]:
%%bash
python convert_surface_vtk.py \
    --input_dir data/subjects_label \
    --cb_dir data/checkerboard_deformed \
    --output_dir data/outputs

## References

- Downloading example data: http://neuroimaging-data-science.org/content/006-image/003-registration.html
- ANTs registration: https://github.com/ANTsX/ANTs/wiki/Anatomy-of-an-antsRegistration-call
- OpenNeuro dataset: https://github.com/OpenNeuroDatasets/ds001233
- MNI152 template: https://nist.mni.mcgill.ca/icbm-152-nonlinear-atlases-2009/
- templateflow: https://www.templateflow.org/
- ndslib: https://pypi.org/project/ndslib/
- FSL: https://fsl.fmrib.ox.ac.uk/fsl/fslwiki